In [9]:
import pandas as pd
df_4users_candidates = pd.read_csv("../data/retrieved/sports_outdoors_5core/ver_0.4_sports_outdoors_candidates.csv")
df_4users_reranked = pd.read_csv("../data/reranked/sports_outdoors_reranked_azure_gpt5.2_sample4_prompt_v08.csv")
df_4users_ground_truth = pd.read_csv("../data/processed/sports_and_outdoors_global_temporal_split/test.csv", names=['user_id', 'item_id', 'rating', 'timestamp'])

df_combined = pd.merge(df_4users_candidates, df_4users_reranked, on=['user_id', 'item_id'], suffixes=('_candidates', '_reranked'))
df_combined = pd.merge(df_combined, df_4users_ground_truth, on=['user_id', 'item_id'], how='left')
df_combined.drop(columns=['score', 'timestamp'], inplace=True)
print(df_combined)


                          user_id     item_id  rank_candidates  rank_reranked  \
0    AE22K4SIBCPAPJIYOJXRNV2SXH7Q  B09MJKJYLQ                1              1   
1    AE22K4SIBCPAPJIYOJXRNV2SXH7Q  B0BS4B6HJQ                2             11   
2    AE22K4SIBCPAPJIYOJXRNV2SXH7Q  B017V088TG                3             27   
3    AE22K4SIBCPAPJIYOJXRNV2SXH7Q  B08KTQRTXS                4             12   
4    AE22K4SIBCPAPJIYOJXRNV2SXH7Q  B0B8T4M8KY                5             17   
..                            ...         ...              ...            ...   
115  AE24ZJ6LKEXXPAPX4OCBF5ZQAVJA  B0BTNZ41Y7               26             30   
116  AE24ZJ6LKEXXPAPX4OCBF5ZQAVJA  B0BP97G4L1               27             13   
117  AE24ZJ6LKEXXPAPX4OCBF5ZQAVJA  B09RQ6PK7V               28             23   
118  AE24ZJ6LKEXXPAPX4OCBF5ZQAVJA  B08GP5NSJ8               29             17   
119  AE24ZJ6LKEXXPAPX4OCBF5ZQAVJA  B0BX4ZDSTQ               30             16   

     rating  
0       NaN  

In [10]:
import numpy as np

# Compute NDCG and MRR per user for a given rank column

def compute_metrics(df: pd.DataFrame, rank_col: str):
    def user_metrics(group: pd.DataFrame):
        ordered = group.sort_values(rank_col)
        rel = ordered["rating"].notna().astype(int)
        ranks = ordered[rank_col]
        dcg = (rel / np.log2(ranks + 1)).sum()
        ideal_rel = rel.sort_values(ascending=False)
        ideal_ranks = np.arange(1, len(ideal_rel) + 1)
        idcg = (ideal_rel.to_numpy() / np.log2(ideal_ranks + 1)).sum()
        ndcg = dcg / idcg if idcg > 0 else 0.0
        hit_ranks = ordered.loc[ordered["rating"].notna(), rank_col]
        rr = 1 / hit_ranks.min() if not hit_ranks.empty else 0.0
        return pd.Series({"ndcg": ndcg, "rr": rr})

    per_user = df.groupby("user_id").apply(user_metrics)
    return per_user["ndcg"].mean(), per_user["rr"].mean()

ndcg_candidates, mrr_candidates = compute_metrics(df_combined, "rank_candidates")
ndcg_reranked, mrr_reranked = compute_metrics(df_combined, "rank_reranked")

print(f"NDCG candidates: {ndcg_candidates:.4f}, MRR candidates: {mrr_candidates:.4f}")
print(f"NDCG reranked: {ndcg_reranked:.4f}, MRR reranked: {mrr_reranked:.4f}")


NDCG candidates: 0.3026, MRR candidates: 0.1122
NDCG reranked: 0.4300, MRR reranked: 0.2682


C:\Users\U762387\AppData\Local\Temp\ipykernel_15844\1093491011.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  per_user = df.groupby("user_id").apply(user_metrics)
C:\Users\U762387\AppData\Local\Temp\ipykernel_15844\1093491011.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  per_user = df.groupby("user_id").apply(user_metrics)
